In [1]:
import random   #코인 토스를 위해서 랜덤을 import

# 5-1 MC업데이트


In [4]:
class GridWorld(): #그리드 월드 클래스 정의
    def __init__(self):
        self.x=0  # 초기 x, y 좌표를 0,0 즉 s_0로 설정해줌
        self.y=0

    def step(self, a): # 매 스텝에 따라 상하좌우로 이동한다
        if a==0:
            self.move_right()  
        elif a==1:
            self.move_left()
        elif a==2:
            self.move_up()
        elif a==3:
            self.move_down()
        reward = -1 # 리워드는 -1로 고정 환경설정이라 지정은 해줬지만 실제로는 관측되기 전까지 모르는 값에 해당
        done = self.is_done()  # step이 terminal state에 도달했는지 확인함
        return (self.x, self.y), reward, done      # (state의 좌표) = (x, y)의 튜플, 리워드, done을 리턴

    def move_right(self):  # 오른쪽을 선택하면 오른쪽 state로 이동
        self.y += 1
        if self.y > 3:    # 오른쪽벽에서 오른쪽으로 state변화가 일어난다면,
            self.y = 3    # 제자리에 머문다 (state방문+1, 에피소드 길이도 +1, 보상도 -1 받음)
    
    def move_left(self):  # x, y의 이동을 제한함으로 인해 환경을 4 *4 그리드로 설정해주었다.
        self.y -= 1
        if self.y < 0:
            self.y = 0

    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0

    def move_down(self):
        self.x += 1
        if self.x > 3:
            self.x = 3

    def is_done(self):
        if self.x == 3 and self.y ==3: # 좌표 3.3 (index=0)에서 시작했으므로 우하단 칸에 들어가면 terminal state로 간주
            return True                # 불리언으로 결정 True를 반환한다.
        else:
            return False
    
    def get_state(self): # 현재 state가 뭔지 알려줌 ( = 실제로는 모르는 값에 해당, 전이확률에 따라 up,right,down,left를 입력 받더라도 다른 state를 가질 수 있다.)
        return (self.x, self.y) 

    def reset(self):   # 처음 리셋을 위함
        self.x = 0 
        self.y = 0
        return (self.x, self.y)    

In [5]:
class Agent (): # 에이전트 클래스를 정의
    def __init__(self):
        pass

    def select_action(self): # 에이전트에 포함된 정책 π
        coin = random.random() # 각 25% 확률로(랜덤) 상,하,좌,우 를 결정해서 
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action # 0,1,2,3으로 코딩된 action을 리턴


In [ ]:
def main():
    env = GridWorld() # 환경 설정
    agent = Agent()  # 에이전트 (+ 정책 포함)
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]] # 리턴값 테이블을 만들어줌
    gamma = 1.0 # 감쇠인자
    alpha = 0.0001 # 업데이트 가중평균치

    for k in range(50000): # 총 5만 번의 에피소드 진행
        done = False
        history = []
        while not done: # terminal state가 아니면
            action = agent.select_action()  # 랜덤(coin토스)으로 action을 결정해주고
            (x,y), reward, done = env.step(action) # 상태변화, reward, terminal state인지 아닌지를 받음
            history.append((x,y,reward)) # (x,y좌표, reward) 를 튜플로 append
        env.reset()
        cum_reward = 0   # 누적 리워드 = 리턴
        for transition in history[::-1]:
            # 방문했던 상태들을 뒤에서부터확인하며 차례차례 역순으로 리턴을 계산, [::-1] 리스트 인덱싱을 사용(역순 접근)
            x, y, reward = transition
            data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y]) # x,y state에 기존 가치함수값과 가중치를 더한 새로운 리턴을 추가해줌. (1-0.0001)은 거의 1이므로 알파만 곱해준거 같다.(아마도, 연산리소스 절약을 위한거인듯)
            cum_reward = cum_reward + gamma*reward # 리턴 계산식 = 리워드 + 감쇠인가(이후 리워드들)

    # 학습이 끝나고 난 후 데이터를 출력해보는 코드
    
    for row in data:
        print(row)

In [ ]:
main()

[-60.97069528045291, -58.83812725103779, -56.576954810005375, -54.1364811519732]
[-59.67388713683955, -55.873217943693334, -51.479086863157335, -47.074672444524495]
[-56.68670417847033, -51.63362314455851, -42.65028397278723, -31.252977887275925]
[-54.79325695014831, -48.20509967075206, -32.3476619387113, 0.0]


# 5.2 TD

In [2]:
def main():
    env  = GridWorld()   #가져오는 조건은 같음, MDP를 모르는 상태니 (실제로 존재하는) reward와 전이확률도 모르는 게 맞음
    agent = Agent()  # 기본 정책도 변화없음 코인토스로 25% 상하좌우로 이동함
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]] # 리턴값 테이블을 만들어줌
    gamma = 1.0 # 감쇠인자
    alpha = 0.01 # 업데이트 가중평균치, MC에 비해 큰 값을 사용 (초기 변화율에는 편차가 크지만 그만큼 변화에 시간이 걸려서 인 것 같다.)

    for k in range(50000): #에피소드 5만번
        done = False # done = True로 마지막에 되어있으니 초기화
        while not done:
            x, y = env.get_state() #좌표 = 변화없음
            action = agent.select_action()
            (x_prime, y_prime), reward, done = env.step(action) #한번 step을 거침 -> (x, y)변화, reward받음, done 판단
            x_prime, y_prime = env.get_state()

            data[x][y] = data[x][y] + alpha*(reward+gamma*data[x_prime][y_prime]-data[x][y]) #(x,y) state의 상태 가치를 reward와 (x', y') 상태가치를 통해 업데이트 해줌
        env.reset() #시작 state로 이동

    # 학습 종료 후 데이터 출력
    for row in data:
        print(row)


In [6]:
main()

[-59.76020845652443, -57.4122112803595, -54.51269066124383, -51.82582014543628]
[-57.646891582398766, -54.864150149525976, -50.59115967610504, -46.120652919129746]
[-54.98804573760294, -50.2679382536611, -40.15974641266741, -29.403401000659724]
[-51.753743375677075, -44.46147980201992, -29.526844822472704, 0]
